In [1]:
#setup based on this: https://t-redactyl.io/blog/2020/08/reading-s3-data-into-a-spark-dataframe-using-sagemaker.html
import boto3
import json 
import time
import pandas as pd
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, isnan, when, count, col
import matplotlib.pyplot as plt
import sagemaker_pyspark
import botocore.session

## Set Spark Session Configuration

In [2]:
session = botocore.session.get_session()
credentials = session.get_credentials()

In [3]:
client = boto3.client('secretsmanager')
response = client.get_secret_value(
    SecretId='sapient-s3-access'
)
response = json.loads(response['SecretString'])
access_key = response["aws_access_key_id"]
secret_key = response["aws_secret_access_key"]

In [4]:
conf = (SparkConf()
        .set("spark.driver.extraClassPath", ":".join(sagemaker_pyspark.classpath_jars())))

In [5]:
spark = (
    SparkSession
    .builder
    .config(conf=conf) \
    .config('fs.s3a.access.key', access_key)
    .config('fs.s3a.secret.key', secret_key)
    .config('spark.network.timeout', 300)
    .config('spark.memory.offHeap.size','4g')
    .config('spark.executor.memory', '16g')
    .appName("sapient")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/22 20:07:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/22 20:07:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Functions to Load and Read Data

In [6]:
# read from raw bucket + write to refined bucket + aggregate final to the trusted bucket
s3_url_raw = "s3a://sapient-bucket-raw/"
s3_url_refined = "s3a://sapient-bucket-refined/"
s3_url_trusted = "s3a://sapient-bucket-trusted/"
bro_cols_conn = ['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 
                 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents']
bro_cols_rep = ['ts', 'level', 'message', 'location']

In [7]:
def readCheckpoint(type='ecar-bro', env='dev', size='small'):
    """
    type: ecar, ecar-bro, bro
    """
    s3_parquet_loc = f"{s3_url_refined}/{size}/{env}/{type}"
    start_time = time.time()
    df = spark.read.parquet(s3_parquet_loc)
    # rdd = spark.sparkContext.parallelize(df.take(1000))
    # print(f"Your dataframe has {rdd.count():,} rows.")
    print("--- %s seconds ---" % (time.time() - start_time))
    return df

In [12]:
df = readCheckpoint('ecar-bro', 'dev', 'small')

--- 0.45285868644714355 seconds ---


In [13]:
df.printSchema()

root
 |-- action: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- object: string (nullable = true)
 |-- objectID: string (nullable = true)
 |-- pid: long (nullable = true)
 |-- ppid: long (nullable = true)
 |-- principal: string (nullable = true)
 |-- tid: long (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- acuity_level: string (nullable = true)
 |-- bro_uid: string (nullable = true)
 |-- dest_ip: string (nullable = true)
 |-- dest_port: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- l4protocol: string (nullable = true)
 |-- src_ip: string (nullable = true)
 |-- src_port: string (nullable = true)



In [14]:
df = readCheckpoint('ecar', 'dev', 'small')

--- 0.5135655403137207 seconds ---


In [15]:
df.printSchema()

root
 |-- action: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- object: string (nullable = true)
 |-- objectID: string (nullable = true)
 |-- pid: long (nullable = true)
 |-- ppid: long (nullable = true)
 |-- principal: string (nullable = true)
 |-- tid: long (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- acuity_level: string (nullable = true)
 |-- base_address: string (nullable = true)
 |-- command_line: string (nullable = true)
 |-- context_info: string (nullable = true)
 |-- data: string (nullable = true)
 |-- dest_ip: string (nullable = true)
 |-- dest_port: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- info_class: string (nullable = true)
 |-- key: string (nullable = true)
 |-- l4protocol: string (nullable = true)
 |-- logo